# Tema: Creación de APIs con FastAPI
## Teoría y Ejemplos


## 1. ¿Qué es FastAPI?

**FastAPI** es un framework moderno y de alto rendimiento para construir APIs (Application Programming Interfaces) con Python 3.7+.

### Características principales:
- **Rápido:** Uno de los frameworks más rápidos de Python, comparable a NodeJS y Go
- **Fácil:** Diseño intuitivo y fácil de aprender
- **Robusto:** Código listo para producción con validación automática
- **Documentación automática:** Genera documentación interactiva (Swagger UI y ReDoc)
- **Type hints:** Usa anotaciones de tipo de Python para validación y autocompletado
- **Basado en estándares:** OpenAPI (Swagger) y JSON Schema

### ¿Qué es una API?
Una **API REST** (Representational State Transfer) es un conjunto de reglas que permite que diferentes aplicaciones se comuniquen entre sí a través de HTTP.

### Analogía:
Una API es como un **camarero en un restaurante**:
- El **cliente** hace un pedido (request)
- El **camarero** (API) lleva el pedido a la cocina
- La **cocina** (servidor) prepara el pedido
- El **camarero** trae la comida (response) al cliente

### Instalación:
```bash
pip install fastapi
pip install "uvicorn[standard]"  # Servidor ASGI
```

## 2. Primera API con FastAPI

Vamos a crear una API simple paso a paso.

In [ ]:
# Importar FastAPI
from fastapi import FastAPI

# Crear instancia de la aplicación
app = FastAPI()

# Definir una ruta (endpoint)
@app.get("/")
def raiz():
    """Endpoint raíz que retorna un saludo."""
    return {"mensaje": "¡Hola Mundo!"}

@app.get("/saludo")
def saludar():
    """Endpoint que retorna un saludo personalizado."""
    return {"mensaje": "Bienvenido a FastAPI"}

# Para ejecutar:
# Guardar como main.py y ejecutar: uvicorn main:app --reload
# Visitar: http://127.0.0.1:8000
# Documentación: http://127.0.0.1:8000/docs

### Estructura básica explicada:

```python
@app.get("/")           # Decorador que define el método HTTP y la ruta
def raiz():             # Función que maneja la petición
    return {...}        # Respuesta (FastAPI la convierte a JSON automáticamente)
```

### Métodos HTTP principales:
- **GET:** Obtener datos (leer)
- **POST:** Crear nuevos recursos
- **PUT:** Actualizar recursos completos
- **PATCH:** Actualizar recursos parcialmente
- **DELETE:** Eliminar recursos

## 3. Path Parameters (Parámetros de Ruta)

Los parámetros de ruta permiten capturar valores directamente desde la URL.

In [ ]:
from fastapi import FastAPI

app = FastAPI()

# Parámetro de ruta simple
@app.get("/usuarios/{usuario_id}")
def obtener_usuario(usuario_id: int):
    """
    Obtiene un usuario por su ID.
    
    - **usuario_id**: ID del usuario (entero)
    """
    return {
        "usuario_id": usuario_id,
        "nombre": f"Usuario {usuario_id}"
    }

# URL: http://127.0.0.1:8000/usuarios/123
# Respuesta: {"usuario_id": 123, "nombre": "Usuario 123"}

In [ ]:
# Múltiples parámetros de ruta
@app.get("/productos/{categoria}/{producto_id}")
def obtener_producto(categoria: str, producto_id: int):
    """
    Obtiene un producto por categoría y ID.
    """
    return {
        "categoria": categoria,
        "producto_id": producto_id,
        "nombre": f"Producto {producto_id} de {categoria}"
    }

# URL: http://127.0.0.1:8000/productos/electronica/456

In [ ]:
# Validación de parámetros con Path
from fastapi import Path

@app.get("/items/{item_id}")
def leer_item(
    item_id: int = Path(
        ...,  # ... significa "requerido"
        title="ID del Item",
        description="El ID debe ser un número positivo",
        ge=1,  # greater than or equal (>=)
        le=1000  # less than or equal (<=)
    )
):
    """Lee un item validando que el ID esté entre 1 y 1000."""
    return {"item_id": item_id}

# Válido: /items/500
# Inválido: /items/0 o /items/1001 (retorna error automáticamente)

## 4. Query Parameters (Parámetros de Consulta)

Los parámetros de consulta van después del `?` en la URL: `/items?skip=0&limit=10`

In [ ]:
from fastapi import FastAPI, Query
from typing import Optional

app = FastAPI()

# Query parameters simples
@app.get("/items/")
def listar_items(skip: int = 0, limit: int = 10):
    """
    Lista items con paginación.
    
    - **skip**: Número de items a saltar (default: 0)
    - **limit**: Número máximo de items a retornar (default: 10)
    """
    return {
        "skip": skip,
        "limit": limit,
        "items": [f"Item {i}" for i in range(skip, skip + limit)]
    }

# URL: http://127.0.0.1:8000/items/?skip=5&limit=3
# Respuesta: {"skip": 5, "limit": 3, "items": ["Item 5", "Item 6", "Item 7"]}

In [ ]:
# Query parameters opcionales
@app.get("/buscar/")
def buscar(
    q: Optional[str] = None,  # Parámetro opcional
    orden: str = "asc"  # Parámetro con valor por defecto
):
    """
    Busca items con filtros opcionales.
    """
    resultado = {"orden": orden}
    
    if q:
        resultado["q"] = q
        resultado["resultados"] = [f"Resultado que contiene '{q}'"]
    else:
        resultado["resultados"] = ["Todos los resultados"]
    
    return resultado

# URL: http://127.0.0.1:8000/buscar/
# URL: http://127.0.0.1:8000/buscar/?q=python
# URL: http://127.0.0.1:8000/buscar/?q=python&orden=desc

In [ ]:
# Validación de query parameters con Query
@app.get("/productos/")
def listar_productos(
    nombre: str = Query(
        ...,  # Requerido
        min_length=3,
        max_length=50,
        description="Nombre del producto a buscar"
    ),
    precio_min: float = Query(
        0,
        ge=0,  # >= 0
        description="Precio mínimo"
    ),
    precio_max: float = Query(
        None,
        le=10000,  # <= 10000
        description="Precio máximo"
    )
):
    """
    Busca productos con validación de parámetros.
    """
    return {
        "nombre": nombre,
        "precio_min": precio_min,
        "precio_max": precio_max
    }

# URL válida: /productos/?nombre=laptop&precio_min=500&precio_max=2000
# URL inválida: /productos/?nombre=pc (nombre muy corto)

## 5. Request Body (Cuerpo de la Petición)

Para enviar datos más complejos (POST, PUT, PATCH), usamos modelos Pydantic.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel, Field, EmailStr
from typing import Optional

app = FastAPI()

# Definir modelo con Pydantic
class Usuario(BaseModel):
    """Modelo de Usuario."""
    nombre: str = Field(..., min_length=2, max_length=50)
    email: str
    edad: int = Field(..., ge=18, le=120)
    activo: bool = True
    descripcion: Optional[str] = None
    
    class Config:
        schema_extra = {
            "example": {
                "nombre": "Ana García",
                "email": "ana@example.com",
                "edad": 25,
                "activo": True,
                "descripcion": "Desarrolladora Python"
            }
        }

@app.post("/usuarios/")
def crear_usuario(usuario: Usuario):
    """
    Crea un nuevo usuario.
    
    FastAPI valida automáticamente que:
    - nombre tenga entre 2 y 50 caracteres
    - edad esté entre 18 y 120
    - todos los campos requeridos estén presentes
    """
    return {
        "mensaje": "Usuario creado exitosamente",
        "usuario": usuario
    }

# Prueba con: POST http://127.0.0.1:8000/usuarios/
# Body (JSON):
# {
#     "nombre": "Ana García",
#     "email": "ana@example.com",
#     "edad": 25
# }

In [ ]:
# Modelo anidado
from typing import List

class Direccion(BaseModel):
    """Modelo de Dirección."""
    calle: str
    ciudad: str
    codigo_postal: str
    pais: str = "España"

class UsuarioCompleto(BaseModel):
    """Modelo de Usuario Completo con dirección."""
    nombre: str
    email: str
    edad: int
    direccion: Direccion  # Modelo anidado
    hobbies: List[str] = []  # Lista de strings

@app.post("/usuarios-completo/")
def crear_usuario_completo(usuario: UsuarioCompleto):
    """Crea un usuario con información completa."""
    return {
        "mensaje": "Usuario completo creado",
        "usuario": usuario
    }

# Body (JSON):
# {
#     "nombre": "Juan Pérez",
#     "email": "juan@example.com",
#     "edad": 30,
#     "direccion": {
#         "calle": "Calle Mayor 123",
#         "ciudad": "Madrid",
#         "codigo_postal": "28001"
#     },
#     "hobbies": ["programar", "leer", "viajar"]
# }

## 6. Métodos HTTP Completos (CRUD)

Ejemplo completo de un CRUD (Create, Read, Update, Delete) para gestionar usuarios.

In [ ]:
from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel
from typing import Dict, Optional

app = FastAPI(title="API de Usuarios", version="1.0.0")

# Modelo de Usuario
class Usuario(BaseModel):
    nombre: str
    email: str
    edad: int
    activo: bool = True

class UsuarioActualizar(BaseModel):
    nombre: Optional[str] = None
    email: Optional[str] = None
    edad: Optional[int] = None
    activo: Optional[bool] = None

# Base de datos simulada
usuarios_db: Dict[int, Usuario] = {}
contador_id = 1


# CREATE - Crear usuario
@app.post(
    "/usuarios/",
    response_model=Dict,
    status_code=status.HTTP_201_CREATED,
    tags=["Usuarios"]
)
def crear_usuario(usuario: Usuario):
    """
    Crea un nuevo usuario.
    
    - **nombre**: Nombre completo del usuario
    - **email**: Email del usuario
    - **edad**: Edad del usuario
    - **activo**: Estado del usuario (default: True)
    """
    global contador_id
    
    usuarios_db[contador_id] = usuario
    usuario_creado = {
        "id": contador_id,
        **usuario.dict()
    }
    contador_id += 1
    
    return {
        "mensaje": "Usuario creado exitosamente",
        "usuario": usuario_creado
    }


# READ - Obtener todos los usuarios
@app.get("/usuarios/", tags=["Usuarios"])
def listar_usuarios(
    skip: int = 0,
    limit: int = 10,
    activo: Optional[bool] = None
):
    """
    Lista todos los usuarios con paginación y filtros opcionales.
    """
    usuarios = []
    
    for id, usuario in usuarios_db.items():
        # Filtrar por estado si se especifica
        if activo is not None and usuario.activo != activo:
            continue
        
        usuarios.append({
            "id": id,
            **usuario.dict()
        })
    
    # Aplicar paginación
    usuarios_paginados = usuarios[skip: skip + limit]
    
    return {
        "total": len(usuarios),
        "skip": skip,
        "limit": limit,
        "usuarios": usuarios_paginados
    }


# READ - Obtener un usuario específico
@app.get("/usuarios/{usuario_id}", tags=["Usuarios"])
def obtener_usuario(usuario_id: int):
    """
    Obtiene un usuario por su ID.
    """
    if usuario_id not in usuarios_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Usuario con ID {usuario_id} no encontrado"
        )
    
    return {
        "id": usuario_id,
        **usuarios_db[usuario_id].dict()
    }


# UPDATE - Actualizar usuario completo
@app.put("/usuarios/{usuario_id}", tags=["Usuarios"])
def actualizar_usuario(usuario_id: int, usuario: Usuario):
    """
    Actualiza todos los datos de un usuario.
    """
    if usuario_id not in usuarios_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Usuario con ID {usuario_id} no encontrado"
        )
    
    usuarios_db[usuario_id] = usuario
    
    return {
        "mensaje": "Usuario actualizado exitosamente",
        "usuario": {
            "id": usuario_id,
            **usuario.dict()
        }
    }


# UPDATE - Actualizar parcialmente
@app.patch("/usuarios/{usuario_id}", tags=["Usuarios"])
def actualizar_parcial_usuario(usuario_id: int, usuario: UsuarioActualizar):
    """
    Actualiza parcialmente los datos de un usuario.
    Solo actualiza los campos proporcionados.
    """
    if usuario_id not in usuarios_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Usuario con ID {usuario_id} no encontrado"
        )
    
    usuario_actual = usuarios_db[usuario_id]
    
    # Actualizar solo los campos proporcionados
    datos_actualizados = usuario.dict(exclude_unset=True)
    usuario_actualizado = usuario_actual.copy(update=datos_actualizados)
    
    usuarios_db[usuario_id] = usuario_actualizado
    
    return {
        "mensaje": "Usuario actualizado parcialmente",
        "usuario": {
            "id": usuario_id,
            **usuario_actualizado.dict()
        }
    }


# DELETE - Eliminar usuario
@app.delete(
    "/usuarios/{usuario_id}",
    status_code=status.HTTP_204_NO_CONTENT,
    tags=["Usuarios"]
)
def eliminar_usuario(usuario_id: int):
    """
    Elimina un usuario por su ID.
    """
    if usuario_id not in usuarios_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Usuario con ID {usuario_id} no encontrado"
        )
    
    del usuarios_db[usuario_id]
    
    return None  # 204 No Content no retorna body


# Endpoint adicional: Estadísticas
@app.get("/usuarios/estadisticas/resumen", tags=["Estadísticas"])
def obtener_estadisticas():
    """
    Obtiene estadísticas generales de los usuarios.
    """
    total_usuarios = len(usuarios_db)
    usuarios_activos = sum(1 for u in usuarios_db.values() if u.activo)
    usuarios_inactivos = total_usuarios - usuarios_activos
    
    if total_usuarios > 0:
        edad_promedio = sum(u.edad for u in usuarios_db.values()) / total_usuarios
    else:
        edad_promedio = 0
    
    return {
        "total_usuarios": total_usuarios,
        "usuarios_activos": usuarios_activos,
        "usuarios_inactivos": usuarios_inactivos,
        "edad_promedio": round(edad_promedio, 2)
    }

## 7. Manejo de Errores y Excepciones

In [ ]:
from fastapi import FastAPI, HTTPException, status
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError

app = FastAPI()

# Lanzar excepción HTTP
@app.get("/items/{item_id}")
def obtener_item(item_id: int):
    if item_id not in range(1, 101):
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} no encontrado",
            headers={"X-Error": "Item no existe"}
        )
    return {"item_id": item_id}


# Manejador de excepción personalizado
class ItemNoEncontrado(Exception):
    def __init__(self, item_id: int):
        self.item_id = item_id

@app.exception_handler(ItemNoEncontrado)
async def manejador_item_no_encontrado(request, exc: ItemNoEncontrado):
    return JSONResponse(
        status_code=404,
        content={
            "error": "Item no encontrado",
            "item_id": exc.item_id,
            "mensaje": f"El item {exc.item_id} no existe en la base de datos"
        }
    )

@app.get("/productos/{producto_id}")
def obtener_producto(producto_id: int):
    if producto_id > 100:
        raise ItemNoEncontrado(item_id=producto_id)
    return {"producto_id": producto_id}


# Manejador global de errores de validación
@app.exception_handler(RequestValidationError)
async def manejador_validacion(request, exc: RequestValidationError):
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={
            "error": "Error de validación",
            "detalle": str(exc),
            "mensaje": "Los datos proporcionados no son válidos"
        }
    )

## 8. Response Models y Status Codes

Controlar exactamente qué se retorna y con qué código de estado.

In [ ]:
from fastapi import FastAPI, status
from pydantic import BaseModel
from typing import List

app = FastAPI()

# Modelos de entrada y salida diferentes
class UsuarioEntrada(BaseModel):
    """Modelo para crear usuario (incluye contraseña)."""
    nombre: str
    email: str
    password: str

class UsuarioSalida(BaseModel):
    """Modelo de respuesta (sin contraseña)."""
    id: int
    nombre: str
    email: str
    
    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "nombre": "Ana García",
                "email": "ana@example.com"
            }
        }


@app.post(
    "/usuarios/",
    response_model=UsuarioSalida,  # Especifica el modelo de respuesta
    status_code=status.HTTP_201_CREATED  # Código de estado
)
def crear_usuario(usuario: UsuarioEntrada):
    """
    Crea un usuario. La contraseña no se incluye en la respuesta.
    """
    # Aquí se guardaría en la BD con la contraseña hasheada
    usuario_guardado = {
        "id": 1,
        "nombre": usuario.nombre,
        "email": usuario.email,
        # password no se incluye en la respuesta
    }
    return usuario_guardado


# Lista de usuarios
@app.get(
    "/usuarios/",
    response_model=List[UsuarioSalida],  # Lista de usuarios
    status_code=status.HTTP_200_OK
)
def listar_usuarios():
    """Lista todos los usuarios."""
    return [
        {"id": 1, "nombre": "Ana", "email": "ana@example.com"},
        {"id": 2, "nombre": "Juan", "email": "juan@example.com"}
    ]

In [ ]:
# Códigos de estado comunes

@app.get("/ok", status_code=status.HTTP_200_OK)
def ok():
    """200 OK - Solicitud exitosa."""
    return {"mensaje": "Todo OK"}

@app.post("/crear", status_code=status.HTTP_201_CREATED)
def crear():
    """201 Created - Recurso creado exitosamente."""
    return {"mensaje": "Recurso creado"}

@app.delete("/eliminar", status_code=status.HTTP_204_NO_CONTENT)
def eliminar():
    """204 No Content - Eliminación exitosa, sin contenido."""
    return None

@app.get("/no-autorizado", status_code=status.HTTP_401_UNAUTHORIZED)
def no_autorizado():
    """401 Unauthorized - No autenticado."""
    return {"error": "No autenticado"}

@app.get("/prohibido", status_code=status.HTTP_403_FORBIDDEN)
def prohibido():
    """403 Forbidden - Sin permisos."""
    return {"error": "Sin permisos"}

@app.get("/no-encontrado", status_code=status.HTTP_404_NOT_FOUND)
def no_encontrado():
    """404 Not Found - Recurso no encontrado."""
    return {"error": "No encontrado"}

@app.post("/validacion", status_code=status.HTTP_422_UNPROCESSABLE_ENTITY)
def error_validacion():
    """422 Unprocessable Entity - Error de validación."""
    return {"error": "Datos inválidos"}

@app.get("/error-servidor", status_code=status.HTTP_500_INTERNAL_SERVER_ERROR)
def error_servidor():
    """500 Internal Server Error - Error del servidor."""
    return {"error": "Error interno"}

## 9. Dependencias (Dependency Injection)

Las dependencias permiten reutilizar lógica común entre endpoints.

In [ ]:
from fastapi import FastAPI, Depends, HTTPException, status
from typing import Optional

app = FastAPI()

# Dependencia simple: parámetros de paginación
def parametros_paginacion(
    skip: int = 0,
    limit: int = 10
) -> dict:
    """Dependencia para manejar paginación."""
    if limit > 100:
        raise HTTPException(
            status_code=400,
            detail="El límite máximo es 100"
        )
    return {"skip": skip, "limit": limit}


@app.get("/items/")
def listar_items(paginacion: dict = Depends(parametros_paginacion)):
    """
    Lista items usando la dependencia de paginación.
    """
    return {
        "paginacion": paginacion,
        "items": [f"Item {i}" for i in range(
            paginacion["skip"],
            paginacion["skip"] + paginacion["limit"]
        )]
    }


@app.get("/productos/")
def listar_productos(paginacion: dict = Depends(parametros_paginacion)):
    """
    Lista productos usando la misma dependencia.
    """
    return {
        "paginacion": paginacion,
        "productos": [f"Producto {i}" for i in range(
            paginacion["skip"],
            paginacion["skip"] + paginacion["limit"]
        )]
    }

In [ ]:
# Dependencia de autenticación simulada
from fastapi import Header

def verificar_token(x_token: str = Header(...)):
    """
    Verifica que el token sea válido.
    Se espera en el header: X-Token
    """
    if x_token != "token-secreto-123":
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token inválido"
        )
    return x_token


@app.get("/protegido/")
def endpoint_protegido(token: str = Depends(verificar_token)):
    """
    Endpoint que requiere autenticación.
    
    Para acceder, incluir header: X-Token: token-secreto-123
    """
    return {
        "mensaje": "Acceso permitido",
        "token_usado": token
    }


# Dependencia con clase
class VerificadorPermisos:
    """Verificador de permisos reutilizable."""
    
    def __init__(self, permisos_requeridos: List[str]):
        self.permisos_requeridos = permisos_requeridos
    
    def __call__(self, x_token: str = Header(...)):
        # Simulación: extraer permisos del token
        permisos_usuario = ["leer", "escribir"]  # Simulado
        
        for permiso in self.permisos_requeridos:
            if permiso not in permisos_usuario:
                raise HTTPException(
                    status_code=status.HTTP_403_FORBIDDEN,
                    detail=f"Permiso '{permiso}' requerido"
                )
        
        return permisos_usuario


@app.post("/admin/crear")
def crear_admin(
    permisos = Depends(VerificadorPermisos(["admin", "escribir"]))
):
    """Endpoint que requiere permisos de admin."""
    return {"mensaje": "Recurso creado con permisos admin"}

## 10. Middleware y CORS

In [ ]:
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import time

app = FastAPI()

# Configurar CORS (permitir peticiones desde otros dominios)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Permitir todos los orígenes (en producción, especificar dominios)
    allow_credentials=True,
    allow_methods=["*"],  # Permitir todos los métodos (GET, POST, etc.)
    allow_headers=["*"],  # Permitir todos los headers
)

# En producción, especificar orígenes:
# allow_origins=[
#     "https://mi-frontend.com",
#     "https://www.mi-frontend.com",
# ]


# Middleware personalizado para medir tiempo de respuesta
@app.middleware("http")
async def agregar_tiempo_proceso(request: Request, call_next):
    """
    Middleware que mide el tiempo de procesamiento de cada petición.
    """
    inicio = time.time()
    
    # Procesar la petición
    response = await call_next(request)
    
    # Calcular tiempo
    tiempo_proceso = time.time() - inicio
    
    # Agregar header personalizado
    response.headers["X-Process-Time"] = str(tiempo_proceso)
    
    return response


@app.get("/")
def raiz():
    """Endpoint de prueba para ver el tiempo de proceso."""
    return {"mensaje": "Revisa el header X-Process-Time"}

## 11. Subir Archivos

In [ ]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from typing import List
import shutil
from pathlib import Path

app = FastAPI()

# Directorio para guardar archivos
UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(exist_ok=True)


# Subir un archivo
@app.post("/upload/")
async def subir_archivo(archivo: UploadFile = File(...)):
    """
    Sube un archivo al servidor.
    
    - **archivo**: Archivo a subir
    """
    # Validar tipo de archivo
    tipos_permitidos = ["image/jpeg", "image/png", "application/pdf"]
    if archivo.content_type not in tipos_permitidos:
        raise HTTPException(
            status_code=400,
            detail=f"Tipo de archivo no permitido. Permitidos: {tipos_permitidos}"
        )
    
    # Validar tamaño (máximo 5MB)
    max_size = 5 * 1024 * 1024  # 5MB
    contenido = await archivo.read()
    if len(contenido) > max_size:
        raise HTTPException(
            status_code=400,
            detail="El archivo es demasiado grande (máximo 5MB)"
        )
    
    # Guardar archivo
    ruta_archivo = UPLOAD_DIR / archivo.filename
    with open(ruta_archivo, "wb") as f:
        f.write(contenido)
    
    return {
        "mensaje": "Archivo subido exitosamente",
        "nombre_archivo": archivo.filename,
        "tipo": archivo.content_type,
        "tamaño": len(contenido)
    }


# Subir múltiples archivos
@app.post("/upload-multiple/")
async def subir_multiples_archivos(archivos: List[UploadFile] = File(...)):
    """
    Sube múltiples archivos al servidor.
    """
    archivos_guardados = []
    
    for archivo in archivos:
        contenido = await archivo.read()
        ruta_archivo = UPLOAD_DIR / archivo.filename
        
        with open(ruta_archivo, "wb") as f:
            f.write(contenido)
        
        archivos_guardados.append({
            "nombre": archivo.filename,
            "tamaño": len(contenido)
        })
    
    return {
        "mensaje": f"{len(archivos_guardados)} archivos subidos",
        "archivos": archivos_guardados
    }

## 12. Conexión a Base de Datos (SQLAlchemy)

Ejemplo de integración con SQLAlchemy y SQLite.

In [ ]:
# Instalación necesaria:
# pip install sqlalchemy

from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
from pydantic import BaseModel
from typing import List

# Configuración de base de datos
SQLALCHEMY_DATABASE_URL = "sqlite:///./usuarios.db"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
    connect_args={"check_same_thread": False}  # Solo para SQLite
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

# Modelo de base de datos
class UsuarioDB(Base):
    """Modelo de tabla Usuario en la base de datos."""
    __tablename__ = "usuarios"
    
    id = Column(Integer, primary_key=True, index=True)
    nombre = Column(String, index=True)
    email = Column(String, unique=True, index=True)
    edad = Column(Integer)

# Crear tablas
Base.metadata.create_all(bind=engine)

# Modelos Pydantic para API
class UsuarioCrear(BaseModel):
    """Modelo para crear usuario."""
    nombre: str
    email: str
    edad: int

class UsuarioRespuesta(BaseModel):
    """Modelo de respuesta."""
    id: int
    nombre: str
    email: str
    edad: int
    
    class Config:
        orm_mode = True  # Permite convertir desde modelo SQLAlchemy

# FastAPI app
app = FastAPI()

# Dependencia para obtener sesión de BD
def obtener_db():
    """Dependencia que proporciona una sesión de base de datos."""
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


# Endpoints
@app.post("/usuarios/", response_model=UsuarioRespuesta)
def crear_usuario(
    usuario: UsuarioCrear,
    db: Session = Depends(obtener_db)
):
    """
    Crea un nuevo usuario en la base de datos.
    """
    # Verificar si el email ya existe
    usuario_existente = db.query(UsuarioDB).filter(
        UsuarioDB.email == usuario.email
    ).first()
    
    if usuario_existente:
        raise HTTPException(
            status_code=400,
            detail="El email ya está registrado"
        )
    
    # Crear usuario
    db_usuario = UsuarioDB(**usuario.dict())
    db.add(db_usuario)
    db.commit()
    db.refresh(db_usuario)
    
    return db_usuario


@app.get("/usuarios/", response_model=List[UsuarioRespuesta])
def listar_usuarios(
    skip: int = 0,
    limit: int = 10,
    db: Session = Depends(obtener_db)
):
    """
    Lista todos los usuarios de la base de datos.
    """
    usuarios = db.query(UsuarioDB).offset(skip).limit(limit).all()
    return usuarios


@app.get("/usuarios/{usuario_id}", response_model=UsuarioRespuesta)
def obtener_usuario(
    usuario_id: int,
    db: Session = Depends(obtener_db)
):
    """
    Obtiene un usuario por ID.
    """
    usuario = db.query(UsuarioDB).filter(UsuarioDB.id == usuario_id).first()
    
    if usuario is None:
        raise HTTPException(
            status_code=404,
            detail="Usuario no encontrado"
        )
    
    return usuario


@app.put("/usuarios/{usuario_id}", response_model=UsuarioRespuesta)
def actualizar_usuario(
    usuario_id: int,
    usuario: UsuarioCrear,
    db: Session = Depends(obtener_db)
):
    """
    Actualiza un usuario existente.
    """
    db_usuario = db.query(UsuarioDB).filter(UsuarioDB.id == usuario_id).first()
    
    if db_usuario is None:
        raise HTTPException(
            status_code=404,
            detail="Usuario no encontrado"
        )
    
    # Actualizar campos
    for key, value in usuario.dict().items():
        setattr(db_usuario, key, value)
    
    db.commit()
    db.refresh(db_usuario)
    
    return db_usuario


@app.delete("/usuarios/{usuario_id}")
def eliminar_usuario(
    usuario_id: int,
    db: Session = Depends(obtener_db)
):
    """
    Elimina un usuario de la base de datos.
    """
    usuario = db.query(UsuarioDB).filter(UsuarioDB.id == usuario_id).first()
    
    if usuario is None:
        raise HTTPException(
            status_code=404,
            detail="Usuario no encontrado"
        )
    
    db.delete(usuario)
    db.commit()
    
    return {"mensaje": "Usuario eliminado exitosamente"}

## 13. Background Tasks (Tareas en Segundo Plano)

In [ ]:
from fastapi import FastAPI, BackgroundTasks
import time

app = FastAPI()

def escribir_log(mensaje: str):
    """
    Función que se ejecuta en segundo plano.
    Simula escribir en un archivo de log.
    """
    time.sleep(2)  # Simula proceso lento
    with open("log.txt", "a") as f:
        f.write(f"{mensaje}\n")
    print(f"Log escrito: {mensaje}")


def enviar_email(email: str, mensaje: str):
    """
    Simula envío de email en segundo plano.
    """
    time.sleep(3)  # Simula tiempo de envío
    print(f"Email enviado a {email}: {mensaje}")


@app.post("/enviar-notificacion/")
async def enviar_notificacion(
    email: str,
    mensaje: str,
    background_tasks: BackgroundTasks
):
    """
    Envía una notificación en segundo plano.
    La respuesta se envía inmediatamente, sin esperar al email.
    """
    # Agregar tareas en segundo plano
    background_tasks.add_task(escribir_log, f"Notificación a {email}")
    background_tasks.add_task(enviar_email, email, mensaje)
    
    return {
        "mensaje": "Notificación programada",
        "email": email,
        "nota": "El email se enviará en segundo plano"
    }


@app.post("/procesar-datos/")
async def procesar_datos(
    datos: dict,
    background_tasks: BackgroundTasks
):
    """
    Procesa datos en segundo plano.
    """
    def procesar():
        time.sleep(5)  # Proceso largo
        print(f"Datos procesados: {datos}")
    
    background_tasks.add_task(procesar)
    
    return {
        "mensaje": "Procesamiento iniciado en segundo plano",
        "datos_recibidos": datos
    }

## 14. Testing de la API

In [ ]:
# Instalación necesaria:
# pip install pytest httpx

# Archivo: test_main.py
"""
from fastapi.testclient import TestClient
from main import app  # Importar la app de tu archivo principal

# Crear cliente de prueba
client = TestClient(app)


def test_leer_raiz():
    '''Prueba el endpoint raíz.'''
    response = client.get("/")
    assert response.status_code == 200
    assert response.json() == {"mensaje": "¡Hola Mundo!"}


def test_crear_usuario():
    '''Prueba crear un usuario.'''
    usuario_datos = {
        "nombre": "Test User",
        "email": "test@example.com",
        "edad": 25
    }
    
    response = client.post("/usuarios/", json=usuario_datos)
    assert response.status_code == 201
    
    data = response.json()
    assert data["usuario"]["nombre"] == "Test User"
    assert data["usuario"]["email"] == "test@example.com"


def test_obtener_usuario_no_existente():
    '''Prueba obtener un usuario que no existe.'''
    response = client.get("/usuarios/9999")
    assert response.status_code == 404


def test_validacion_edad():
    '''Prueba validación de edad.'''
    usuario_invalido = {
        "nombre": "Usuario Inválido",
        "email": "invalido@example.com",
        "edad": 15  # Menor de 18
    }
    
    response = client.post("/usuarios/", json=usuario_invalido)
    assert response.status_code == 422  # Unprocessable Entity


# Ejecutar pruebas:
# pytest test_main.py -v
"""

print("Tests de ejemplo creados. Guarda en test_main.py y ejecuta con pytest")

## 15. Ejemplo Completo: API de Blog

Aplicación completa con posts y comentarios.

In [ ]:
from fastapi import FastAPI, HTTPException, status, Depends
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime

app = FastAPI(
    title="Blog API",
    description="API completa para un blog con posts y comentarios",
    version="1.0.0"
)

# ========== MODELOS ==========

class ComentarioBase(BaseModel):
    """Modelo base de comentario."""
    contenido: str = Field(..., min_length=1, max_length=500)
    autor: str = Field(..., min_length=2, max_length=50)

class Comentario(ComentarioBase):
    """Modelo completo de comentario."""
    id: int
    post_id: int
    fecha_creacion: datetime

class PostBase(BaseModel):
    """Modelo base de post."""
    titulo: str = Field(..., min_length=5, max_length=200)
    contenido: str = Field(..., min_length=10)
    autor: str = Field(..., min_length=2, max_length=50)
    etiquetas: List[str] = []

class PostCrear(PostBase):
    """Modelo para crear post."""
    pass

class Post(PostBase):
    """Modelo completo de post."""
    id: int
    fecha_creacion: datetime
    fecha_actualizacion: Optional[datetime] = None
    comentarios: List[Comentario] = []
    
    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "titulo": "Mi primer post",
                "contenido": "Este es el contenido del post...",
                "autor": "Ana García",
                "etiquetas": ["python", "fastapi"],
                "fecha_creacion": "2024-01-01T10:00:00",
                "comentarios": []
            }
        }

# ========== BASE DE DATOS SIMULADA ==========

posts_db: dict[int, dict] = {}
comentarios_db: dict[int, dict] = {}
post_id_counter = 1
comentario_id_counter = 1


# ========== ENDPOINTS DE POSTS ==========

@app.post(
    "/posts/",
    response_model=Post,
    status_code=status.HTTP_201_CREATED,
    tags=["Posts"]
)
def crear_post(post: PostCrear):
    """
    Crea un nuevo post en el blog.
    """
    global post_id_counter
    
    post_data = {
        "id": post_id_counter,
        **post.dict(),
        "fecha_creacion": datetime.now(),
        "fecha_actualizacion": None,
        "comentarios": []
    }
    
    posts_db[post_id_counter] = post_data
    post_id_counter += 1
    
    return post_data


@app.get(
    "/posts/",
    response_model=List[Post],
    tags=["Posts"]
)
def listar_posts(
    skip: int = 0,
    limit: int = 10,
    etiqueta: Optional[str] = None,
    autor: Optional[str] = None
):
    """
    Lista todos los posts con filtros opcionales.
    
    - **skip**: Número de posts a saltar
    - **limit**: Número máximo de posts a retornar
    - **etiqueta**: Filtrar por etiqueta
    - **autor**: Filtrar por autor
    """
    posts = list(posts_db.values())
    
    # Aplicar filtros
    if etiqueta:
        posts = [p for p in posts if etiqueta in p["etiquetas"]]
    
    if autor:
        posts = [p for p in posts if autor.lower() in p["autor"].lower()]
    
    # Aplicar paginación
    return posts[skip: skip + limit]


@app.get(
    "/posts/{post_id}",
    response_model=Post,
    tags=["Posts"]
)
def obtener_post(post_id: int):
    """
    Obtiene un post específico por su ID.
    """
    if post_id not in posts_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Post con ID {post_id} no encontrado"
        )
    
    return posts_db[post_id]


@app.put(
    "/posts/{post_id}",
    response_model=Post,
    tags=["Posts"]
)
def actualizar_post(post_id: int, post: PostCrear):
    """
    Actualiza un post existente.
    """
    if post_id not in posts_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Post con ID {post_id} no encontrado"
        )
    
    post_actual = posts_db[post_id]
    
    # Actualizar campos
    posts_db[post_id] = {
        "id": post_id,
        **post.dict(),
        "fecha_creacion": post_actual["fecha_creacion"],
        "fecha_actualizacion": datetime.now(),
        "comentarios": post_actual["comentarios"]
    }
    
    return posts_db[post_id]


@app.delete(
    "/posts/{post_id}",
    status_code=status.HTTP_204_NO_CONTENT,
    tags=["Posts"]
)
def eliminar_post(post_id: int):
    """
    Elimina un post y todos sus comentarios.
    """
    if post_id not in posts_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Post con ID {post_id} no encontrado"
        )
    
    # Eliminar comentarios asociados
    comentarios_a_eliminar = [
        c_id for c_id, c in comentarios_db.items()
        if c["post_id"] == post_id
    ]
    
    for c_id in comentarios_a_eliminar:
        del comentarios_db[c_id]
    
    # Eliminar post
    del posts_db[post_id]
    
    return None


# ========== ENDPOINTS DE COMENTARIOS ==========

@app.post(
    "/posts/{post_id}/comentarios/",
    response_model=Comentario,
    status_code=status.HTTP_201_CREATED,
    tags=["Comentarios"]
)
def crear_comentario(post_id: int, comentario: ComentarioBase):
    """
    Crea un comentario en un post.
    """
    global comentario_id_counter
    
    # Verificar que el post existe
    if post_id not in posts_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Post con ID {post_id} no encontrado"
        )
    
    # Crear comentario
    comentario_data = {
        "id": comentario_id_counter,
        "post_id": post_id,
        **comentario.dict(),
        "fecha_creacion": datetime.now()
    }
    
    comentarios_db[comentario_id_counter] = comentario_data
    comentario_id_counter += 1
    
    # Agregar comentario al post
    posts_db[post_id]["comentarios"].append(comentario_data)
    
    return comentario_data


@app.get(
    "/posts/{post_id}/comentarios/",
    response_model=List[Comentario],
    tags=["Comentarios"]
)
def listar_comentarios(post_id: int):
    """
    Lista todos los comentarios de un post.
    """
    if post_id not in posts_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Post con ID {post_id} no encontrado"
        )
    
    return posts_db[post_id]["comentarios"]


@app.delete(
    "/comentarios/{comentario_id}",
    status_code=status.HTTP_204_NO_CONTENT,
    tags=["Comentarios"]
)
def eliminar_comentario(comentario_id: int):
    """
    Elimina un comentario.
    """
    if comentario_id not in comentarios_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Comentario con ID {comentario_id} no encontrado"
        )
    
    # Obtener post_id del comentario
    post_id = comentarios_db[comentario_id]["post_id"]
    
    # Eliminar comentario del post
    posts_db[post_id]["comentarios"] = [
        c for c in posts_db[post_id]["comentarios"]
        if c["id"] != comentario_id
    ]
    
    # Eliminar comentario
    del comentarios_db[comentario_id]
    
    return None


# ========== ENDPOINTS DE ESTADÍSTICAS ==========

@app.get(
    "/estadisticas/",
    tags=["Estadísticas"]
)
def obtener_estadisticas():
    """
    Obtiene estadísticas generales del blog.
    """
    total_posts = len(posts_db)
    total_comentarios = len(comentarios_db)
    
    # Etiquetas más usadas
    todas_etiquetas = []
    for post in posts_db.values():
        todas_etiquetas.extend(post["etiquetas"])
    
    from collections import Counter
    etiquetas_populares = Counter(todas_etiquetas).most_common(5)
    
    # Autores más activos
    autores = [post["autor"] for post in posts_db.values()]
    autores_activos = Counter(autores).most_common(5)
    
    return {
        "total_posts": total_posts,
        "total_comentarios": total_comentarios,
        "promedio_comentarios_por_post": (
            total_comentarios / total_posts if total_posts > 0 else 0
        ),
        "etiquetas_populares": [
            {"etiqueta": e, "cantidad": c}
            for e, c in etiquetas_populares
        ],
        "autores_activos": [
            {"autor": a, "posts": c}
            for a, c in autores_activos
        ]
    }


# ========== ENDPOINT RAÍZ ==========

@app.get("/", tags=["General"])
def raiz():
    """
    Endpoint raíz con información de la API.
    """
    return {
        "mensaje": "Bienvenido a la API del Blog",
        "version": "1.0.0",
        "documentacion": "/docs",
        "endpoints": {
            "posts": "/posts/",
            "comentarios": "/posts/{post_id}/comentarios/",
            "estadisticas": "/estadisticas/"
        }
    }

## 16. Despliegue y Buenas Prácticas

### Estructura de Proyecto Recomendada:

```
mi-api/
│
├── app/
│   ├── __init__.py
│   ├── main.py              # Aplicación principal
│   ├── models.py            # Modelos Pydantic
│   ├── database.py          # Configuración de BD
│   ├── schemas.py           # Esquemas de datos
│   ├── crud.py              # Operaciones de BD
│   ├── dependencies.py      # Dependencias
│   └── routers/             # Routers por recurso
│       ├── __init__.py
│       ├── usuarios.py
│       ├── posts.py
│       └── comentarios.py
│
├── tests/
│   ├── __init__.py
│   └── test_main.py
│
├── .env                     # Variables de entorno
├── .gitignore
├── requirements.txt         # Dependencias
└── README.md
```

### Archivo requirements.txt:
```
fastapi==0.104.1
uvicorn[standard]==0.24.0
pydantic==2.5.0
python-multipart==0.0.6
sqlalchemy==2.0.23
python-dotenv==1.0.0
```

### Variables de Entorno (.env):
```
DATABASE_URL=sqlite:///./app.db
SECRET_KEY=tu-clave-secreta-aqui
ENVIRONMENT=development
```

### Comandos para Ejecutar:

```bash
# Desarrollo (con auto-reload)
uvicorn app.main:app --reload --port 8000

# Producción
uvicorn app.main:app --host 0.0.0.0 --port 8000 --workers 4

# Con Gunicorn (producción)
gunicorn app.main:app --workers 4 --worker-class uvicorn.workers.UvicornWorker --bind 0.0.0.0:8000
```

## 📚 Recursos Adicionales

### Documentación Oficial:
- [FastAPI Docs](https://fastapi.tiangolo.com/)
- [Pydantic Docs](https://docs.pydantic.dev/)
- [SQLAlchemy Docs](https://docs.sqlalchemy.org/)

### Herramientas Útiles:
- **Postman:** Cliente para probar APIs
- **Insomnia:** Alternativa a Postman
- **HTTPie:** Cliente HTTP de línea de comandos

### Próximos Pasos:
1. Autenticación con JWT
2. WebSockets para comunicación en tiempo real
3. GraphQL con Strawberry
4. Caché con Redis
5. Rate limiting
6. Containerización con Docker
7. CI/CD con GitHub Actions